In [ ]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Trabalho final da disciplina
# Raspagem de documentos de Cota para o Exercício da Atividade Parlamentar de um deputado
# Prof. Alvaro Justen
#

In [30]:
#!pip install wget

In [31]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import wget

In [ ]:
# Faz a requisicao na pagina com os dados - uma página de Cota para o Exercício da Atividade Parlamentar

In [32]:
req = urlopen('https://www.camara.leg.br/cota-parlamentar/analitico?nuDeputadoId=3402&dataInicio=4/2021&dataFim=4/2021&despesa=5&nomeHospede=&nomePassageiro=&nomeFornecedor=&cnpjFornecedor=&numDocumento=&sguf=')

In [ ]:
# Faz o parseamento para facilitar a leitura

In [33]:
soup = BeautifulSoup(req.read(), 'html.parser')

In [ ]:
# Separa o ponto exata onde esta a tabela

In [34]:
tables_ele = soup.findAll('table', {'class': 'tabela-2'})

In [35]:
gastos = []

In [36]:
# Iteracao
for table_ele in tables_ele:
  # Iteracao dentro dos elementos tr
  for row in table_ele.findAll('tr'):
    #print(row)
    # Procura os itens td
    cols = row.findAll('td')
    # Acessa apenas os não vazios
    if len(cols) != 0:
      # Acessa menos o rodapé
      if cols[0].text.strip() != "Total":
        # Captura cada item correspondente na tabela
        dicionario = {"nome_fornecedor": cols[0].text.strip(),
              "cpf_cnpj": cols[1].text.strip(),
              "data_emissao": cols[2].text.strip(),
              "numero_documento": cols[3].text.strip(),
              "link_documento": "https://www.camara.leg.br" + cols[3].find('a').get('href').strip(),
              "valor_despesa": cols[4].text.strip(),
              "deducoes_desembolso": cols[6].text.strip(),
              }
        # Adiciona na lista
        gastos.append(dicionario)
       

In [37]:
# Cria o dataframe final
df = pd.DataFrame(gastos)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   nome_fornecedor      6 non-null      object
 1   cpf_cnpj             6 non-null      object
 2   data_emissao         6 non-null      object
 3   numero_documento     6 non-null      object
 4   link_documento       6 non-null      object
 5   valor_despesa        6 non-null      object
 6   deducoes_desembolso  6 non-null      object
dtypes: object(7)
memory usage: 464.0+ bytes


In [38]:
df

,nome_fornecedor,cpf_cnpj,data_emissao,numero_documento,link_documento,valor_despesa,deducoes_desembolso
0,BM Gestão de Mídias Sociais,36.785.758/0001-20,01/05/2021,14,https://www.camara.leg.br/cota-parlamentar/doc...,"R$6.000,00","R$6.000,00"
1,BITTAR EDITORA PLANEJAMENTO ESTRATEGICO LTDA,39.572.622/0001-20,08/04/2021,17,https://www.camara.leg.br/cota-parlamentar/not...,"R$5.000,00","R$5.000,00"
2,INOVATUM SOLUCOES EM TECNOLOGIA DA INFORMACAO ...,15.022.614/0001-41,10/05/2021,5,https://www.camara.leg.br/cota-parlamentar/not...,"R$2.000,00","R$2.000,00"
3,NATALI AMORIM GOMES MORALES 39260832802,40.753.621/0001-60,07/05/2021,00000005,https://www.camara.leg.br/cota-parlamentar/doc...,"R$2.000,00","R$2.000,00"
4,Lucas Santos da Silva,41.101.476/0001-04,07/05/2021,002114699,https://www.camara.leg.br/cota-parlamentar/doc...,"R$1.500,00","R$1.500,00"
5,TAMISA BARBOSA DE OLIVEIRA 04090320186,41.191.298/0001-41,06/05/2021,002114175,https://www.camara.leg.br/cota-parlamentar/doc...,"R$1.500,00","R$1.500,00"


In [10]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
# Salva em CSV

In [114]:
df.to_csv('/drive/My Drive/Data-driven/Insper/cota_parlamentar.csv', index=False)

In [ ]:
# Baixa os documentos nos links, se forem PDFs

In [41]:
[wget.download(x) for x in df['link_documento'] if '.pdf' in x]

['7185522.pdf', '7188973.pdf', '7188975.pdf', '7188974.pdf']